# Worksheet 07

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.2576734228484785, 4.7578070307302065, 4.494763776310731, 5.519291023630238, 4.263312078189034, 4.3494647666655455, 5.7824554854558325, 4.266697898963812, 4.222110372702063, 6.483269648901544]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(c2)

[8.278296849945427, 5.813667150936817, 7.995181874025521, 9.745541343767743, 8.320641880519243, 8.427090877057903, 7.594882820270209, 8.515202260310884, 8.116089468934934, 8.139445157309503]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [5]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop(-1)
        data.append(p1)
    else:
        p2 = c2.pop(-1)
        data.append(p2)
print(data)

[6.483269648901544, 4.222110372702063, 8.139445157309503, 8.116089468934934, 8.515202260310884, 7.594882820270209, 4.266697898963812, 5.7824554854558325, 8.427090877057903, 4.3494647666655455]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The mean and var and P(c_j) for each component

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [7]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.222110372702063, 4.266697898963812, 5.7824554854558325, 4.3494647666655455]
[6.483269648901544, 8.139445157309503, 8.116089468934934, 8.515202260310884, 7.594882820270209, 8.427090877057903]
P(C_1) = 0.4,  P(C_2) = 0.6
mean_1 = 4.655182130946813,  mean_2 = 7.879330038797495
var_1 = 0.4256698672244311,  var_2 = 0.4763308588376638


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [8]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(  pdf_i[0] *  prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] *  prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  6.483269648901544
probability of observing that point if it came from cluster 0 =  9.264834295607407e-05
probability of observing that point if it came from cluster 1 =  0.011421310854414196
point =  4.222110372702063
probability of observing that point if it came from cluster 0 =  0.5585632972915255
probability of observing that point if it came from cluster 1 =  1.3247883858812547e-13
point =  8.139445157309503
probability of observing that point if it came from cluster 0 =  2.6480749212321912e-15
probability of observing that point if it came from cluster 1 =  0.7215179314388418
point =  8.116089468934934
probability of observing that point if it came from cluster 0 =  4.1430942581485364e-15
probability of observing that point if it came from cluster 1 =  0.7402077116267254
point =  8.515202260310884
probability of observing that point if it came from cluster 0 =  1.3052534315187455e-18
probability of observing that point if it came from cluster 1 =  0.34358324897239106
poi

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [26]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum(map(lambda x : (x[0] * x[1]), zip(prob_c0_x, ((data-mean[0])*np.transpose(data-mean[0]))))) / sum(prob_c0_x)  , sum(map(lambda x : (x[0] * x[1]), zip(prob_c1_x, ((data - mean[1])*np.transpose(data - mean[1]))))) / sum(prob_c0_x) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.40026190113585536,  P(C_2) = 0.5997380988641446
mean_1 = 4.656861507583312,  mean_2 = 7.879617193330967
var_1 = 0.42900328285735456,  var_2 = 0.7144412203262696


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [27]:
new_prob_c0_x = [] # P(C_0 | X_i)
new_prob_c1_x = [] # P(C_1 | X_i)
new_prob_x = None # P(X_i)

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    new_prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    new_prob_c0_x.append(  pdf_i[0] *  prob_c[0] / new_prob_x )
    new_prob_c1_x.append( pdf_i[1] *  prob_c[1] / new_prob_x )

probs = zip(data, new_prob_c0_x, new_prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  6.483269648901544
probability of observing that point if it came from cluster 0 =  0.00010781722000981084
probability of observing that point if it came from cluster 1 =  0.08269112503418952
point =  4.222110372702063
probability of observing that point if it came from cluster 0 =  0.5564734965362091
probability of observing that point if it came from cluster 1 =  1.137376080350226e-06
point =  8.139445157309503
probability of observing that point if it came from cluster 0 =  4.5557790968193264e-15
probability of observing that point if it came from cluster 1 =  0.5226644839533485
point =  8.116089468934934
probability of observing that point if it came from cluster 0 =  7.077101726286377e-15
probability of observing that point if it came from cluster 1 =  0.5286329827433982
point =  8.515202260310884
probability of observing that point if it came from cluster 0 =  2.535198709322638e-18
probability of observing that point if it came from cluster 1 =  0.37591182515105775
point 

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [33]:
res = []
probs = zip(data, new_prob_c0_x, new_prob_c1_x)
for d in probs:
    if(d[1]>d[2]):
        res.append((d[0],0))
    else:
        res.append((d[0],1))

res

[(6.483269648901544, 1),
 (4.222110372702063, 0),
 (8.139445157309503, 1),
 (8.116089468934934, 1),
 (8.515202260310884, 1),
 (7.594882820270209, 1),
 (4.266697898963812, 0),
 (5.7824554854558325, 0),
 (8.427090877057903, 1),
 (4.3494647666655455, 0)]

### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.